In [1]:
import pandas as pd
import numpy as np
import xgboost
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from imblearn.combine import SMOTEENN
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split,StratifiedKFold,GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import make_pipeline
import matplotlib.pyplot as plt
import seaborn as sns
np.set_printoptions(threshold=np.inf)
pd.set_option('display.width',None) 
pd.set_option('display.max_columns', None)
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['axes.unicode_minus'] = False
from sklearn.utils import resample
from scipy.optimize import minimize
import warnings 
warnings.filterwarnings ('ignore') 
import random
from sklearn.utils import shuffle
from sklearn.metrics import roc_curve
from sklearn.metrics import auc as auc_func
import scipy.stats as stats
from imblearn.under_sampling import RandomUnderSampler
from sklearn.utils import shuffle

In [1]:
df_train_result=pd.read_csv("./PE0602_out3.csv",index_col="一次就诊号",low_memory=False,encoding="utf-8")
print(df_train_result.shape)
df_train_result.info()

In [2]:
null_counts = df_train_result.isnull().sum()
print(null_counts)

In [6]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
def preprocess_data(data, numeric_cols, excluded_cols=None):
    all_cols = data.columns.tolist()
    if excluded_cols:
        categorical_cols = [col for col in all_cols if col not in numeric_cols and col not in excluded_cols]
    else:
        categorical_cols = [col for col in all_cols if col not in numeric_cols]

    data = data.reset_index(drop=True)
    numeric_data = data[numeric_cols]
    categorical_data = data[categorical_cols]
    encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
    encoded_data = pd.DataFrame()
    feature_names = []
    for col in categorical_cols:
        unique_values = categorical_data[col].unique()
        if len(unique_values) > 2:
            encoded_col = pd.DataFrame(encoder.fit_transform(categorical_data[[col]]),
                                       columns=[f"{col}_{category}" for category in encoder.categories_[0]])
            encoded_data = pd.concat([encoded_data, encoded_col], axis=1)
        else:
            encoded_data = pd.concat([encoded_data, categorical_data[[col]]], axis=1)
        feature_names.extend(encoded_data.columns.tolist())
    processed_data = pd.concat([data[numeric_cols], encoded_data, data[excluded_cols]], axis=1)
    return processed_data

In [7]:
numeric_cols = []
excluded_cols = ['LABEL', '是否前瞻','孕周','年份','身高', '年龄','BMI','孕','产','初潮年龄','孕前体重','月经周期']
df_train_result2 = preprocess_data(df_train_result, numeric_cols, excluded_cols)

In [3]:
colnames_concatenated = ' '.join(df_train_result2.columns)
print(colnames_concatenated)

In [4]:
null_counts = df_train_result2.isnull().sum()
print(null_counts)

In [5]:
df_train_result_1 = df_train_result2.loc[df_train_result2['是否前瞻'] == 0]
df_train_result_1.drop(['是否前瞻','孕周'],axis=1,inplace=True)
print(df_train_result_1.shape)

In [6]:
df_test_result_1 = df_train_result2.loc[df_train_result2['是否前瞻'] == 1]
df_test_result_1.drop(['是否前瞻','孕周'],axis=1,inplace=True)
df_train_result2['PRETREM_PE'] = 0  # 先默认全部赋值为0
df_train_result2['EARLY_PE'] = 0  # 先默认全部赋值为0

df_train_result2.loc[(df_train_result2['孕周'] < 37) & (df_train_result2['LABEL'] == 1), 'PRETREM_PE'] = 1
df_train_result2.loc[(df_train_result2['孕周'] < 34) & (df_train_result2['LABEL'] == 1), 'EARLY_PE'] = 1

df_test_result_2 = df_train_result2.loc[df_train_result2['是否前瞻'] == 1]
df_test_result_2.drop(['是否前瞻','孕周','LABEL'],axis=1,inplace=True)
print(df_test_result_1.shape)
print(df_test_result_2.shape)

In [12]:
y = df_train_result_1['LABEL']
X = df_train_result_1.drop(['LABEL'], axis=1)
oX_test = df_test_result_1.drop('LABEL', axis=1)
oy_test=df_test_result_1['LABEL']
oy_test1=df_test_result_2['PRETREM_PE']
oy_test2=df_test_result_2['EARLY_PE']

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold

# Initialize the random forest classifier
rf = RandomForestClassifier(random_state=42)

kf = KFold(n_splits=5, shuffle=True, random_state=42)
importances_df = pd.DataFrame()
for i, (train_index, test_index) in enumerate(kf.split(X)):
    # Split the data
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]    
    rf.fit(X_train, y_train)    
    importances = rf.feature_importances_    
    fold_importances_df = pd.DataFrame({
        f'Feature': X.columns,
        f'Importance Fold {i+1}': importances
    })

    if importances_df.empty:
        importances_df = fold_importances_df
    else:
        importances_df = pd.merge(importances_df, fold_importances_df, on='Feature')

importances_df['Mean Importance'] = importances_df.filter(like='Importance').mean(axis=1)
importances_df = importances_df.sort_values(by='Mean Importance', ascending=False)
importances_df['Cumulative Importance'] = np.cumsum(importances_df['Mean Importance'])
print(importances_df)

In [ ]:
threshold = 0.85
important_features = importances_df[importances_df['Cumulative Importance'] <= threshold]['Feature'].values.tolist()
print(important_features)

In [13]:
important_features=['BMI', '年龄', '身高', '慢性高血压',
                    '早孕反应','既往子痫', '孕', '糖尿病',
                    '多胎', '月经规律', '既往流产', '疤痕', '死胎',
                    '家族高血压', '肾病', '辅助生殖']

X=X[important_features]
oX_test=oX_test[important_features]

In [14]:
oX_test.shape

(1760, 16)

In [16]:
from sklearn.mixture import GaussianMixture
bic_scores = []
n_components_range = range(2, 38)
for n_components in n_components_range:
    gmm = GaussianMixture(n_components=n_components, random_state=42)
    gmm.fit(X[y == 1])
    bic = gmm.bic(X[y == 1])
    bic_scores.append(bic)
for n_components, bic in zip(n_components_range, bic_scores):
    print(f"n_components={n_components}: BIC={bic}")
best_n_components = n_components_range[bic_scores.index(min(bic_scores))]
print(f"Best n_components: {best_n_components}")

n_components=2: BIC=-15676.806984567458
n_components=3: BIC=-42720.46233649928
n_components=4: BIC=-49909.66335866315
n_components=5: BIC=-51564.31704893525
n_components=6: BIC=-56593.23358043929
n_components=7: BIC=-67978.98724795559
n_components=8: BIC=-68977.27936838832
n_components=9: BIC=-47159.3235823987
n_components=10: BIC=-64214.33600018466
n_components=11: BIC=-70354.09181540518
n_components=12: BIC=-60686.85961229724
n_components=13: BIC=-63525.277810806976
n_components=14: BIC=-77746.61753858456
n_components=15: BIC=-87509.97415689129
n_components=16: BIC=-78565.3009925828
n_components=17: BIC=-79447.7515629926
n_components=18: BIC=-80632.02169440422
n_components=19: BIC=-82551.9139512586
n_components=20: BIC=-77090.43764379542
n_components=21: BIC=-77569.60013645198
n_components=22: BIC=-76873.3245889741
n_components=23: BIC=-71076.45224530264
n_components=24: BIC=-69107.64952631589
n_components=25: BIC=-82493.84870468797
n_components=26: BIC=-53324.30991683515
n_component

# 采样处理

In [17]:
n_components = best_n_components 
gmm = GaussianMixture(n_components=n_components,random_state=42)
gmm.fit(X[y == 1])
weights = gmm.weights_
covariances = gmm.covariances_
means = gmm.means_
alpha=1.84
inverse_weights = 1 / (weights ** alpha)

inverse_weights /= np.sum(inverse_weights)
gmm.weights_=inverse_weights
num_positive_samples = int(np.sum(y == 1) * (1 - min(inverse_weights)))
np.random.seed(42)
new_positive_samples = gmm.sample(num_positive_samples)[0]
X_positive_resampled = np.vstack((X[y == 1].values, new_positive_samples))
y_positive_resampled = np.ones(X_positive_resampled.shape[0])
X_resampled1 = np.vstack((X[y == 0].values, X_positive_resampled))
y_resampled1 = np.hstack((y[y == 0].values, y_positive_resampled))
unique_classes, counts = np.unique(y_resampled1, return_counts=True)
count_class_0 = counts[np.where(unique_classes == 0)[0][0]]
count_class_1 = counts[np.where(unique_classes == 1)[0][0]]
undersample = RandomUnderSampler(sampling_strategy={0: count_class_1*3, 1: count_class_1}, random_state=42)
X_resampled, y_resampled = undersample.fit_resample(X_resampled1, y_resampled1)
X_resampled,y_resampled = shuffle(X_resampled, y_resampled, random_state=42)


In [15]:
num_positive_samples

1632

In [16]:
y_positive_resampled.sum()

3267.0

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)
print(X_train.shape,X_test.shape)

# 模型训练

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier,VotingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn import svm
from sklearn.linear_model import LogisticRegression,LinearRegression
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier


In [19]:
para_lightgbm={'colsample_bytree': 1, 'learning_rate': 0.1, 'min_child_samples': 50, 'n_estimators': 100, 'num_leaves': 127, 'subsample': 0.8,'random_state':42 }
para_adaboost={'learning_rate': 0.07, 'n_estimators': 500,'random_state':42 }
para_RF={'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'auto', 'max_depth': 50,'random_state':42 }
para_gbdt={'learning_rate': 0.1, 'max_depth': 9, 'max_features': 9, 'min_samples_leaf': 3, 'min_samples_split': 8, 'subsample': 0.9,'random_state':42 }
para_xgboost={'colsample_bytree': 0.8, 'learning_rate': 0.03, 'max_depth': 8, 'min_child_weight': 3, 'n_estimators': 200, 'reg_alpha': 0.1, 'reg_lambda': 0, 'subsample': 0.7,'random_state':42 }
para_mlp={'activation': 'logistic', 'alpha': 0.001, 'hidden_layer_sizes': (50,),'random_state':42 }
para_bayes={'priors': [0.4, 0.6] }
para_logist={'C': 1.0,'random_state':42 }
para_svm={"gamma":'auto',"degree":5,"C":0.8,'random_state':42 }

# 运行十个模型

In [20]:
aclf1=AdaBoostClassifier(**para_adaboost)
aclf2=RandomForestClassifier(**para_RF)
aclf3=MLPClassifier(**para_mlp)
aclf4=GradientBoostingClassifier(**para_gbdt)
aclf5=GaussianNB(**para_bayes)
aclf6=xgboost.XGBClassifier(**para_xgboost)
aclf7=LogisticRegression(**para_logist)
aclf8=svm.SVC(**para_svm,probability = True)
aclf9=CatBoostClassifier(verbose=0,random_seed=42)
aclf10=LGBMClassifier(**para_lightgbm )

taclf1=aclf1.fit(X_train, y_train)
taclf2=aclf2.fit(X_train, y_train)
taclf3=aclf3.fit(X_train, y_train)
taclf4=aclf4.fit(X_train, y_train)
taclf5=aclf5.fit(X_train, y_train)
taclf6=aclf6.fit(X_train, y_train)
taclf7=aclf7.fit(X_train, y_train)
taclf8=aclf8.fit(X_train, y_train)
taclf9=aclf9.fit(X_train, y_train)
taclf10=aclf10.fit(X_train, y_train)

In [21]:
def get_tpr_at_fpr(y_true, y_scores, fpr_threshold):
    fpr, tpr, thresholds = roc_curve(y_true, y_scores)
    idx = np.argmin(np.abs(fpr - fpr_threshold))  
    return tpr[idx]  

In [7]:
models = [taclf1, taclf2, taclf3, taclf4, taclf5, taclf6, taclf7, taclf8, taclf9, taclf10]
predictions_test = np.array([model.predict_proba(X_test) for model in models])
accs = []
precs = []
npvs = []
recs = []
specs = []
f1s = []
aucs = []
DR90s = []
DR95s = []
for i, model in enumerate(models):
    p_predict = model.predict(X_test)
    p_predict_proba = predictions_test[i]
    
    acc = accuracy_score(y_test, p_predict)
    prec = precision_score(y_test, p_predict, pos_label=1)
    npv = precision_score(y_test, p_predict, pos_label=0)
    rec = recall_score(y_test, p_predict, pos_label=1)
    spec = recall_score(y_test, p_predict, pos_label=0)
    f1 = f1_score(y_test, p_predict, pos_label=1)
    auc = roc_auc_score(y_test, p_predict_proba[:, 1])
    DR90=get_tpr_at_fpr(y_test,p_predict_proba[:, 1],0.1)
    DR95=get_tpr_at_fpr(y_test,p_predict_proba[:, 1],0.05)
    accs.append(acc)
    precs.append(prec)
    npvs.append(npv)
    recs.append(rec)
    specs.append(spec)
    f1s.append(f1)
    aucs.append(auc)
    DR90s.append(DR90)
    DR95s.append(DR95)  
    
    print(f"内部测试集 Model {i + 1}:  Accuracy: {acc:.4f}, Precision: {prec:.4f}, NPV: {npv:.4f}, Recall: {rec:.4f}, Specificity: {spec:.4f}, F1 score: {f1:.4f}, AUC: {auc:.4f},DR90: {DR90:.4f}, DR95: {DR95:.4f}")

predictions_test = np.array([model.predict_proba(oX_test) for model in models])


accs = []
precs = []
npvs = []
recs = []
specs = []
f1s = []
aucs = []
DR90s = []
DR95s = []

for i, model in enumerate(models):
    p_predict = model.predict(oX_test)
    p_predict_proba = predictions_test[i]    
    acc = accuracy_score(oy_test, p_predict)
    prec = precision_score(oy_test, p_predict, pos_label=1)
    npv = precision_score(oy_test, p_predict, pos_label=0)
    rec = recall_score(oy_test, p_predict, pos_label=1)
    spec = recall_score(oy_test, p_predict, pos_label=0)
    f1 = f1_score(oy_test, p_predict, pos_label=1)
    auc = roc_auc_score(oy_test, p_predict_proba[:, 1])
    DR90=get_tpr_at_fpr(oy_test,p_predict_proba[:, 1],0.1)
    DR95=get_tpr_at_fpr(oy_test,p_predict_proba[:, 1],0.05)
    accs.append(acc)
    precs.append(prec)
    npvs.append(npv)
    recs.append(rec)
    specs.append(spec)
    f1s.append(f1)
    aucs.append(auc)
    DR90s.append(DR90)
    DR95s.append(DR95)    
    print(f"外部测试集 Model {i + 1}:  Accuracy: {acc:.4f}, Precision: {prec:.4f}, NPV: {npv:.4f}, Recall: {rec:.4f}, Specificity: {spec:.4f}, F1 score: {f1:.4f}, AUC: {auc:.4f},DR90: {DR90:.4f}, DR95: {DR95:.4f}")

predictions_test = np.array([model.predict_proba(oX_test) for model in models])


accs = []
precs = []
npvs = []
recs = []
specs = []
f1s = []
aucs = []
DR90s = []
DR95s = []

for i, model in enumerate(models):
    p_predict = model.predict(oX_test)
    p_predict_proba = predictions_test[i]    
    acc = accuracy_score(oy_test1, p_predict)
    prec = precision_score(oy_test1, p_predict, pos_label=1)
    npv = precision_score(oy_test1, p_predict, pos_label=0)
    rec = recall_score(oy_test1, p_predict, pos_label=1)
    spec = recall_score(oy_test1, p_predict, pos_label=0)
    f1 = f1_score(oy_test1, p_predict, pos_label=1)
    auc = roc_auc_score(oy_test1, p_predict_proba[:, 1])
    DR90=get_tpr_at_fpr(oy_test1,p_predict_proba[:, 1],0.1)
    DR95=get_tpr_at_fpr(oy_test1,p_predict_proba[:, 1],0.05)
    # 添加评估指标到相应的列表中
    accs.append(acc)
    precs.append(prec)
    npvs.append(npv)
    recs.append(rec)
    specs.append(spec)
    f1s.append(f1)
    aucs.append(auc)
    DR90s.append(DR90)
    DR95s.append(DR95)    
    print(f"外部测试集preterm Model {i + 1}:  Accuracy: {acc:.4f}, Precision: {prec:.4f}, NPV: {npv:.4f}, Recall: {rec:.4f}, Specificity: {spec:.4f}, F1 score: {f1:.4f}, AUC: {auc:.4f},DR90: {DR90:.4f}, DR95: {DR95:.4f}")

predictions_test = np.array([model.predict_proba(oX_test) for model in models])


accs = []
precs = []
npvs = []
recs = []
specs = []
f1s = []
aucs = []
DR90s = []
DR95s = []

for i, model in enumerate(models):
    p_predict = model.predict(oX_test)
    p_predict_proba = predictions_test[i]    
    acc = accuracy_score(oy_test2, p_predict)
    prec = precision_score(oy_test2, p_predict, pos_label=1)
    npv = precision_score(oy_test2, p_predict, pos_label=0)
    rec = recall_score(oy_test2, p_predict, pos_label=1)
    spec = recall_score(oy_test2, p_predict, pos_label=0)
    f1 = f1_score(oy_test2, p_predict, pos_label=1)
    auc = roc_auc_score(oy_test2, p_predict_proba[:, 1])
    DR90=get_tpr_at_fpr(oy_test2,p_predict_proba[:, 1],0.1)
    DR95=get_tpr_at_fpr(oy_test2,p_predict_proba[:, 1],0.05)
    accs.append(acc)
    precs.append(prec)
    npvs.append(npv)
    recs.append(rec)
    specs.append(spec)
    f1s.append(f1)
    aucs.append(auc)
    DR90s.append(DR90)
    DR95s.append(DR95)    
    print(f"外部测试集EARLY-PE Model {i + 1}:  Accuracy: {acc:.4f}, Precision: {prec:.4f}, NPV: {npv:.4f}, Recall: {rec:.4f}, Specificity: {spec:.4f}, F1 score: {f1:.4f}, AUC: {auc:.4f},DR90: {DR90:.4f}, DR95: {DR95:.4f}")



In [10]:
models = [taclf1, taclf2, taclf3, taclf4, taclf5, taclf6, taclf7, taclf8, taclf9, taclf10]

predictions_test = np.array([model.predict_proba(X_test) for model in models])
accs = []
precs = []
npvs = []
recs = []
specs = []
f1s = []
aucs = []
DR90s = []
DR95s = []
for i, model in enumerate(models):
    p_predict = model.predict(X_test)
    p_predict_proba = predictions_test[i]
    
    acc = accuracy_score(y_test, p_predict)
    prec = precision_score(y_test, p_predict, pos_label=1)
    npv = precision_score(y_test, p_predict, pos_label=0)
    rec = recall_score(y_test, p_predict, pos_label=1)
    spec = recall_score(y_test, p_predict, pos_label=0)
    f1 = f1_score(y_test, p_predict, pos_label=1)
    auc = roc_auc_score(y_test, p_predict_proba[:, 1])
    DR90=get_tpr_at_fpr(y_test,p_predict_proba[:, 1],0.1)
    DR95=get_tpr_at_fpr(y_test,p_predict_proba[:, 1],0.05)
    accs.append(acc)
    precs.append(prec)
    npvs.append(npv)
    recs.append(rec)
    specs.append(spec)
    f1s.append(f1)
    aucs.append(auc)
    DR90s.append(DR90)
    DR95s.append(DR95)  
    
    print(f"内部测试集 Model {i + 1}:  Accuracy: {acc:.4f}, Precision: {prec:.4f}, NPV: {npv:.4f}, Recall: {rec:.4f}, Specificity: {spec:.4f}, F1 score: {f1:.4f}, AUC: {auc:.4f},DR90: {DR90:.4f}, DR95: {DR95:.4f}")


In [24]:
predictions_test = np.array([model.predict_proba(X_test) for model in models])

In [11]:
import matplotlib.pyplot as plt

model_names = ['AdaBoost', 'RF', 'MLP', 'GBDT', 'GNB', 'XGBoost', 'LR', 'SVM', 'CatBoost', 'LightGBM']

plt.figure(figsize=(5, 5),dpi=600)
plt.rcParams.update({'font.size': 8})

for i in range(len(models)):
    p_predict_proba = predictions_test[i]
    fpr, tpr, _ = roc_curve(y_test, p_predict_proba[:, 1], pos_label=1)
    roc_auc = roc_auc_score(y_test, p_predict_proba[:, 1])
    tpr_at_fpv_0_1 = tpr[np.argmin(abs(fpr - 0.1))]
    if tpr_at_fpv_0_1 > 0.72:
        plt.plot(fpr, tpr, lw=2,color='red', label=f'{model_names[i]} (AUC:{roc_auc:.4f}, TPR:{tpr_at_fpv_0_1:.4f})')
    else:
        plt.plot(fpr, tpr, lw=1, linestyle='--',label=f'{model_names[i]} (AUC:{roc_auc:.4f}, TPR:{tpr_at_fpv_0_1:.4f})')

        
plt.axvline(x=0.1, color='grey', linestyle='--', linewidth=1, alpha=0.7)

plt.xticks([0.1 * i for i in range(21)])  # Set x-axis ticks with increments of 0.1
plt.xlim([0.0, 0.2])
plt.ylim([0.0, 0.9])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right",fontsize=6, frameon=False)
plt.show()


In [12]:
n_bootstraps = 100
rng_seed = 42  
alpha = 0.05
import string
letters = string.ascii_uppercase[:10] 
fig, axes = plt.subplots(3, 4, figsize=(16, 11), dpi=600)
axes = axes.ravel()

model_labels = ['AdaBoost', 'RF', 'MLP', 'GBDT', 'GNB', 'XGBoost', 'LR', 'SVM', 'CatBoost', 'LightGBM']
predictions_test = np.array([model.predict_proba(X_test) for model in models])


for i, (model, ax, label) in enumerate(zip(models, axes[:10], model_labels)):
    p_predict_proba = predictions_test[i]
    fpr, tpr, _ = roc_curve(y_test, p_predict_proba[:, 1])
    roc_auc = auc_func(fpr, tpr)

    ax.plot(fpr, tpr, lw=2, label=f' {label} (AUC :{roc_auc:.4f})')
    
    # Bootstrap CI
    tprs = []
    aucs = []
    base_fpr = np.linspace(0, 1, 101)
    np.random.seed(rng_seed)

    for j in range(n_bootstraps):
        y_test_bootstrap = resample(y_test, replace=True, n_samples=len(y_test), random_state=j)
        p_predict_proba_bootstrap = resample(p_predict_proba, replace=True, n_samples=len(p_predict_proba), random_state=j)
        
        if len(np.unique(y_test_bootstrap)) == 2:
            fpr_bs, tpr_bs, _ = roc_curve(y_test_bootstrap, p_predict_proba_bootstrap[:, 1])
            tpr_bs_interp = np.interp(base_fpr, fpr_bs, tpr_bs)
            tpr_bs_interp[0] = 0.0
            tprs.append(tpr_bs_interp)
            
            aucs.append(auc_func(fpr_bs, tpr_bs))
        else:
            continue
    
    tprs = np.array(tprs)
    mean_tprs = tprs.mean(axis=0)
    std_tprs = tprs.std(axis=0)
    
    # Computing the lower and upper bound of the 95% confidence interval
    tprs_upper = np.minimum(mean_tprs + 1.96 * std_tprs, 1)
    tprs_lower = mean_tprs - 1.96 * std_tprs
    
    aucs = np.array(aucs)
    lower_auc = np.percentile(aucs, alpha / 2 * 100)
    upper_auc = np.percentile(aucs, (1 - alpha / 2) * 100)

    ax.fill_between(base_fpr, tprs_lower, tprs_upper, color='grey', alpha=0.2,
                     label=r'95% CI: [{:.2f}-{:.2f}]'.format(lower_auc, upper_auc))

    ax.plot([0, 1], [0, 1], color='silver', lw=1, linestyle='--')
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate', fontsize=12)
    ax.set_ylabel('True Positive Rate', fontsize=12)
    letter_label = letters[i]
    ax.text(0, 1.05, f'({letter_label})   {label} ROC Curve with 95% CI', fontsize=14, transform=ax.transAxes)
    ax.legend(loc="lower right",fontsize=12, frameon=False)

for ax in axes[10:]:
    ax.axis('off')

plt.tight_layout()
